# **Writing Viterbi Algorithm for the Primer**

First we define all the parameters, such as states, emmision matrix, and transmission matrix as given in the primer:


In [1]:
import math

states = ['E', '5', 'I']
emissions = {
    'E': {'A':0.25, 'C':0.25, 'G':0.25, 'T':0.25},
    '5': {'A':0.05, 'C':0.0,  'G':0.95, 'T':0.0 },
    'I': {'A':0.40, 'C':0.10, 'G':0.10, 'T':0.40}
}
start_prob = {'E':1.0, '5':0.0, 'I':0.0}
trans_prob = {
    'E': {'E':0.9, '5':0.1, 'I':0.0},
    '5': {'E':0.0, '5':0.0, 'I':1.0},
    'I': {'E':0.0, '5':0.0, 'I':0.9}
}
end_prob = {'E':0.0, '5':0.0, 'I':0.1}

Now we take log of all probabilities:

In [2]:
def to_log(d):
    return {i: (math.log(p) if p>0 else float('-inf')) for i, p in d.items()}

log_start = to_log(start_prob)
log_end   = to_log(end_prob)
log_emit  = {s: to_log(emissions[s]) for s in states}
log_trans = {s: to_log(trans_prob[s]) for s in states}

Now we write a function to get log probability of given path and sequence:

In [3]:
def get_log_prob_of_path(path, seq):
    logp = log_start[path[0]]
    for i, (st, base) in enumerate(zip(path, seq)):
        logp += log_emit[st][base]
        if i < len(path)-1:
            logp += log_trans[st][path[i+1]]
    logp += log_end[path[-1]]
    return logp

Examples:

In [12]:
seq = 'CTTCATGTGAAAGCAGACGTAAGTCA'
path = 'EEEEEEEEEEEEEEEEEE5IIIIIII'
print("Log-prob is: ", get_log_prob_of_path(path, seq))

Log-prob is:  -41.21967768602254


In [13]:
seq = 'CTTCATGTGAA'
path = 'EEIEEE5E5EE'
print("Log-prob is: ", get_log_prob_of_path(path, seq))

Log-prob is:  -inf


Next we write a function that computes the best path and best log probability given a sequence using Viterbi algorithm:

In [8]:
def viterbi(seq):
    n = len(seq)
    V = [{ } for _ in range(n)]
    backpointer = [{ } for _ in range(n)]

    # Initialization
    for s in states:
        V[0][s] = log_start[s] + log_emit[s][seq[0]]
        backpointer[0][s] = None

    # Recursion
    for t in range(1, n):
        for s in states:
            best_prev, best_score = None, float('-inf')
            for ps in states:
                score = V[t-1][ps] + log_trans[ps][s] + log_emit[s][seq[t]]
                if score > best_score:
                    best_score, best_prev = score, ps
            V[t][s] = best_score
            backpointer[t][s] = best_prev

    # Termination
    best_last, best_score = None, float('-inf')
    for s in states:
        score = V[n-1][s] + log_end[s]
        if score > best_score:
            best_score, best_last = score, s

    if(best_score == float('-inf')):
        return '', float('-inf')

    # Backtrack to get path
    path = [best_last]
    for t in range(n-1, 0, -1):
        path.insert(0, backpointer[t][path[0]])
    return ''.join(path), best_score

Examples:

In [9]:
seq = 'CTTCATGTGAAAGCAGACGTAAGTCA'
best_path, best_score = viterbi(seq)
print("Viterbi best path: ", best_path)
print("Viterbi log-probability: ", best_score)

Viterbi best path:  EEEEEEEEEEEEEEEEEE5IIIIIII
Viterbi log-probability:  -41.21967768602254


In [10]:
seq = 'CTTGATATTAATGC'
best_path, best_score = viterbi(seq)
print("Viterbi best path: ", best_path)
print("Viterbi log-probability: ", best_score)

Viterbi best path:  EEE5IIIIIIIIII
Viterbi log-probability:  -21.90980827695273


In [11]:
seq = 'CCC'
best_path, best_score = viterbi(seq)
print("Viterbi best path: ", best_path)
print("Viterbi log-probability: ", best_score)

Viterbi best path:  
Viterbi log-probability:  -inf
